In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.5 MB 2.5 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.2
    Uninstalling pandas-1.5.2:
      Successfully uninstalled pandas-1.5.2
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 71.5 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 290 kB 19.8 MB/s eta 0:00:01
  A

In [2]:
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import re
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [4]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Load Data

In [5]:
dataset_df = pd.read_pickle("/notebooks/KURI-BERT/data/pe_dataset_w_essay_position_pos_tags_pickle")

In [6]:
dataset_df = dataset_df.drop_duplicates(subset=['label_and_comp_idxs', 'text'], keep='first').reset_index(drop=True)

In [7]:
dataset_df = dataset_df.sort_values(['essay_nr', 'argument_bound_1'], ascending=[1, 1]).reset_index(drop=True)

In [8]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
2,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
3,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
4,essay001,T6,Premise 1088 1191,All of these skills help them to get on well w...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,3,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADP, DET, NOUN, VERB...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5958,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5959,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5960,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5961,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard o

In [9]:
dataset_df['paragraph'][1926] = 'Based on the reasons demonstrated above, I prefer to express my opinion in person because the advantages of this are qualitatively greater than that of the opposite. In other words, I think the best way of communication is by means of face-to-face. It is not to say, of course, that other points of view are completely without merit. Still, I am firmly convinced that the reasons I proposed in favor of my own sentiment are much more tenable and cogent.'

/tmp/ipykernel_157/867583492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][1926] = 'Based on the reasons demonstrated above, I prefer to express my opinion in person because the advantages of this are qualitatively greater than that of the opposite. In other words, I think the best way of communication is by means of face-to-face. It is not to say, of course, that other points of view are completely without merit. Still, I am firmly convinced that the reasons I proposed in favor of my own sentiment are much more tenable and cogent.'


In [10]:
dataset_df['paragraph'][4831] = 'Finally, although I prefer to live in a big city, I can not help but assure that this is a matter of personal taste, to be clear, I would love to spend some days in countryside to recharge my soul and relieve myself from daily stress.'

/tmp/ipykernel_157/3379091247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][4831] = 'Finally, although I prefer to live in a big city, I can not help but assure that this is a matter of personal taste, to be clear, I would love to spend some days in countryside to recharge my soul and relieve myself from daily stress.'


In [11]:
dataset_df['paragraph'][3908] = 'I do believe that great success requires taking great risks. If you want to gain very high profits in investments, you should use great and very high amounts of money in very risky financial decisions and dealings in which you may lose much amount of money. If you want to achieve a remarkable success in an important exam, you should risk studying all the time and sacrificing your free time and your favorite hobbies. The more you take risks, the greater your successes will be. If you don’t take any risks, you will have an ordinary life with average successes. That’s why it is important to take risks.'

/tmp/ipykernel_157/2273872724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][3908] = 'I do believe that great success requires taking great risks. If you want to gain very high profits in investments, you should use great and very high amounts of money in very risky financial decisions and dealings in which you may lose much amount of money. If you want to achieve a remarkable success in an important exam, you should risk studying all the time and sacrificing your free time and your favorite hobbies. The more you take risks, the greater your successes will be. If you don’t take any risks, you will have an ordinary life with average successes. That’s why it is important to take risks.'


In [12]:
dataset_df['paragraph'][4306] = 'In conclusion, experiences that shape my thinking and the way to live my life are teach me to be a better person. Because I could find a better way to solve problems. Besides, experiences could shape me to be more aware about my health. Moreover, these experiences could make me to make plan for my future life. These experiences taught me to change my life to be a better person.'

/tmp/ipykernel_157/1196760580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][4306] = 'In conclusion, experiences that shape my thinking and the way to live my life are teach me to be a better person. Because I could find a better way to solve problems. Besides, experiences could shape me to be more aware about my health. Moreover, these experiences could make me to make plan for my future life. These experiences taught me to change my life to be a better person.'


#### get ams

In [13]:
with open("am_list.txt") as file_am:
    list_of_ams = []
    for line in file_am:
        list_of_ams.append(line.rstrip('\n'))

FileNotFoundError: [Errno 2] No such file or directory: 'am_list.txt'

In [ ]:
def am_in_prefix(list_a, list_b):
    
    equal_flag = 0
    
    for i in range(len(list_a)):
        
        sub_list = list_a[i:i+len(list_b)]
        
        if sub_list == list_b:
            
            equal_flag = 1
            break
        
        else:
            
            equal_flag = 0
            
    return equal_flag

In [ ]:
def get_am(x, list_of_ams):
    
    am = []
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    prefix = re.findall(r'\s|,|[^,\s]+', prefix) 
    prefix = [item for item in prefix if item != " "]
    prefix = [word.casefold() for word in prefix]
    
    list_of_ams = [am.casefold() for am in list_of_ams]
    list_of_ams = list(set(list_of_ams))
  
    for am_phrase in list_of_ams:
        
        am_words_list = re.findall(r'\s|,|[^,\s]+', am_phrase)
        am_words_list = [item for item in am_words_list if item != " "]
        
        if am_in_prefix(prefix, am_words_list) and len(am) == 0:
            
            am.append(am_phrase)
    
    return am         

In [ ]:
dataset_df['am'] = dataset_df.apply(lambda x: get_am(x, list_of_ams), axis=1)

In [ ]:
dataset_df['am']

### do group by for every required column

In [ ]:
paras_df = dataset_df.groupby("paragraph", sort=False)

In [ ]:
paras_df

In [ ]:
df_by_paras_text = paras_df["text"].apply(list)

In [ ]:
df_by_paras_text

In [ ]:
df_by_paras_text = df_by_paras_text.reset_index()

In [ ]:
df_by_paras_text

In [ ]:
df_by_paras_labels = paras_df["label_ComponentType"].apply(list).reset_index()

In [ ]:
# Nice.

In [ ]:
df_by_paras_split = paras_df['split_y'].first()

In [ ]:
df_by_paras_split = df_by_paras_split.reset_index()

In [ ]:
df_by_paras_split

In [ ]:
df_by_paras_ams = paras_df["am"].apply(list)

In [ ]:
df_by_paras_ams

In [ ]:
df_by_paras_ams = df_by_paras_ams.reset_index()

In [ ]:
df_by_paras_ams

In [ ]:
# get essays

In [ ]:
df_by_paras_essay = paras_df['essay_nr'].first()

In [ ]:
df_by_paras_essay = df_by_paras_essay.reset_index()

In [ ]:
df_by_paras_essay

In [ ]:
# make the first two DFs lists of lists

In [ ]:
def make_acs_l_o_l(x):
    
    list_acs = x.text
    
    new_list = [[ac] for ac in list_acs]
    
    return new_list

In [ ]:
df_by_paras_text['paragraph_components_list'] = df_by_paras_text.apply(lambda x: make_acs_l_o_l(x), axis=1)

In [ ]:
# ok now for the second labels df

In [ ]:
def make_labels_l_o_l(x):
    
    list_labels = x.label_ComponentType
    
    new_list = [[label] for label in list_labels]
    
    return new_list

In [ ]:
df_by_paras_labels['paragraph_labels_list'] = df_by_paras_labels.apply(lambda x: make_labels_l_o_l(x), axis=1)

In [ ]:
# ok seems correct.

In [ ]:
# now combine the three dataframes

In [ ]:
df = pd.concat([df_by_paras_text, df_by_paras_labels, df_by_paras_ams, df_by_paras_split, df_by_paras_essay], axis=1)

In [ ]:
df

In [ ]:
df = df.loc[:,~df.columns.duplicated()].copy()

In [ ]:
df

In [ ]:
df = df.drop(columns=['text', 'label_ComponentType'])

In [ ]:
df = df.rename(columns={"am": "paragraph_markers_list"})

In [ ]:
df = df.rename(columns={"split_y": "split"})

In [ ]:
df

In [ ]:
def is_equal(x):
    
    # flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_markers_list
    
    return len(list_ac) == len(list_am)

In [ ]:
df['ac=am'] = df.apply(lambda x: is_equal(x), axis=1)

In [ ]:
df['ac=am'].value_counts()

In [ ]:
def is_equal_label(x):
    
    flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_labels_list
    
    if len(list_ac) == len(list_am):
        
        flag = 1
        
    else:
        
        flag = 0
        
    return flag

In [ ]:
df['ac=lab'] = df.apply(lambda x: is_equal_label(x), axis=1)

In [ ]:
df['ac=lab'].value_counts()

In [ ]:
# ok both correct.

In [ ]:
df = df.drop(columns=df.columns[-1])

In [ ]:
df = df.drop(columns=df.columns[-1])

In [ ]:
df

In [ ]:
def get_numeric_labels(x):
    
    numeric_labels = []
    
    string_labels = x.paragraph_labels_list
    
    for label in string_labels:
        
        if label == ['MajorClaim']:
            
            numeric_labels.append(0)
            
        elif label == ['Claim']:
            
            numeric_labels.append(1)
            
        elif label == ['Premise']:
            
            numeric_labels.append(2)
            
    return numeric_labels

In [ ]:
df['paragraph_labels_numeric'] = df.apply(lambda x: get_numeric_labels(x), axis=1)

In [ ]:
# correct the five problematic things in the DF.

In [ ]:
df['paragraph_components_list'][409] = [['when children take jobs, they tend to be more responsible'],
 ['whether they can earn money or not will depend on their effectiveness and attitudes in working'],
 ['To keep their jobs, children will have to try their best to finish their duties'],
 ['if a child works as a shop assistant, he will have to keep an eye on good and products, calculate prices correctly and take responsibilities for any lost']]

In [ ]:
df['paragraph_components_list'][473] = [['children need human interactions'],
 ['No matter how advanced technology will be, it is clear that a computer will never be able to communicate with its pupils'],
 ["As a recent interview with Josh Trumm – the founder of the 'Teachers Global Group'- reveals, human interactions will never be imitated by computers: 'This verbal and non-verbal communication between teachers and children is the first step towards a proper education"],
 ["Without someone who can explain and understand their problems, pupils cannot actually learn'"],
 ['only a teacher can feel and connect with pupils, be a friend and a model']]

In [ ]:
df['paragraph_components_list'][771] = [['the individual should finance his or her education'],
 ['it would cause too much cost from taxpayers and the government']]

In [ ]:
df['paragraph_components_list'][1061] = [['as parents are involved in the same environment'],
 ['they could thus support much more to their children, such as domain knowledge and industry-related information'],
 ['my father is a software programmer, and he indeed teaches me a lot in programming and some computer knowledge'],
 ['Whenever I met a difficulty, I would reach him for consulting'],
 ['he has many experiences'],
 ['I could save enormous time in finding solution'],
 ['This truly makes me more competitive than my colleagues'],
 ['If I worked in the different field from my father, I would have faced much difficulty than I do now']]

In [ ]:
df['paragraph_components_list'][1470] = [['With rapid development of the internet, people are able to enjoy quick electronic communication via internet'],
 ['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],
 ['mobile phones have shortened the distance of communication'],
 ['mobile phones have shortened the distance of communication']]

In [ ]:
df['paragraph_components_list'][1470] = [['With rapid development of the internet, people are able to enjoy quick electronic communication via internet'],
  ['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],
  ['mobile phones have shortened the distance of communication']]

In [ ]:
df['paragraph_markers_list'][1470] = [[], ['for instance'], ['therefore,']]

In [ ]:
df

In [ ]:
# get ACs

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def get_ac_spans(x):
    
    paragraph = x.paragraph
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    ac_list = x.paragraph_components_list
    
    ac_spans = []        
                
    for ac in ac_list:            

        ac_tokenized = tokenizer.tokenize(ac[0])
        
        for i in range(len(paragraph_tokenized)):
            
            span = (i, i+len(ac_tokenized)-1)
            
            if paragraph_tokenized[span[0]:span[1]+1] == ac_tokenized and span not in ac_spans:
                
                ac_spans.append(span)    
                 
    return ac_spans

In [ ]:
df['ac_spans'] = df.apply(lambda x: get_ac_spans(x), axis=1)

In [ ]:
ac_spans_list = df['ac_spans'].tolist()

In [ ]:
def get_inter_span_intervals(ac_spans):
    
    if ac_spans[0][0] == 0:
        
        ac_inter_spans = [(0, 0)]
        
    else:
        
        ac_inter_spans = [(0, ac_spans[0][0]-1)]
      
    ac_inter_spans += [(ac_spans[i][1] + 1, ac_spans[i+1][0]-1) for i in range(len(ac_spans)-1)]
    
    return ac_inter_spans

In [ ]:
def get_am_spans(x):
    
    
    df_idx = df.index[df['paragraph'] == x.paragraph]
    # print(df_idx[0])
    paragraph = x.paragraph
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    am_list = x.paragraph_markers_list
    
    ac_spans = x.ac_spans
    ac_inter_spans = get_inter_span_intervals(ac_spans)
    # print(ac_inter_spans)
    am_spans = [(-1, -1)] * len(am_list)
                
    for idx, am in enumerate(am_list):
        inter_span =  ac_inter_spans[idx] 
        
        if am:
            
            am_tokenized = tokenizer.tokenize(am[0])

            for i in range(inter_span[0], inter_span[1]):

                ac_prefix = paragraph_tokenized[inter_span[0]:inter_span[1]]

                span = (i, i+len(am_tokenized)-1)

                if paragraph_tokenized[span[0]:span[1]+1] == am_tokenized:
                
                    am_spans[idx] = span
    
    return am_spans

In [ ]:
df['am_spans'] = df.apply(lambda x: get_am_spans(x), axis=1)

In [ ]:
# correct! 79 to 387 is the correction work we need.
# SANITY CHECK BELOW

In [ ]:
df['am_spans'], df['ac_spans']

In [ ]:
def sanity_check(x):
    
    return len(x.ac_spans) == len(x.am_spans) 

In [ ]:
df['sanity'] = df.apply(lambda x: sanity_check(x), axis=1)

In [ ]:
df['sanity'].value_counts()

In [ ]:
# nice! correct. now all am_spans and ac_spans are equal in length.

In [ ]:
df = df.drop(columns=df.columns[-1])

In [ ]:
df

In [ ]:
df = df.rename(columns={'am_spans' : 'paragraph_am_spans', 'ac_spans' : 'paragraph_ac_spans', 'paragraph_labels_numeric' : 'paragraph_labels'})

In [ ]:
df

#### dataset

In [ ]:
train_df = df[df.split=='TRAIN'].reset_index(drop=True)
test_df = df[df.split=='TEST'].reset_index(drop=True)

In [ ]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [ ]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [ ]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [ ]:
dataset

In [ ]:
torch.save(dataset, os.path.join("/notebooks/KURI-BERT/data", 'pe_dataset_for_bert_minus.pt'))